In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np

# Load the MovieTweetings dataset

## urls
url_movies = 'https://github.com/sidooms/MovieTweetings/blob/master/latest/movies.dat?raw=true'
url_users = 'https://github.com/sidooms/MovieTweetings/blob/master/latest/users.dat?raw=true'
url_ratings = 'https://github.com/sidooms/MovieTweetings/blob/master/latest/ratings.dat?raw=true'

## read csv
movies = pd.read_csv(url_movies, delimiter='::', header=None, names=['movie_id', 'title', 'genres'], engine='python')
users = pd.read_csv(url_users, delimiter='::', header=None, names=['user_id'], engine='python')
ratings = pd.read_csv(url_ratings, delimiter='::', header=None, names=['user_id', 'movie_id', 'rating', 'timestamp'], engine='python')

## Merge dataframes to get movie titles in the ratings dataframe
#movie_ratings = pd.merge(ratings, movies, on='movie_id')


## Looking at the data

In [2]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38018 entries, 0 to 38017
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   movie_id  38018 non-null  int64 
 1   title     38018 non-null  object
 2   genres    37964 non-null  object
dtypes: int64(1), object(2)
memory usage: 891.2+ KB


In [3]:
users.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 71707 entries, 1 to 71707
Data columns (total 1 columns):
 #   Column   Non-Null Count  Dtype
---  ------   --------------  -----
 0   user_id  71707 non-null  int64
dtypes: int64(1)
memory usage: 1.1 MB


In [4]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 921398 entries, 0 to 921397
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype
---  ------     --------------   -----
 0   user_id    921398 non-null  int64
 1   movie_id   921398 non-null  int64
 2   rating     921398 non-null  int64
 3   timestamp  921398 non-null  int64
dtypes: int64(4)
memory usage: 28.1 MB


In [5]:
# Use this cell to find the following information:
# number of movies
print("Number of movies", movies["movie_id"].nunique())

# number of ratings
print("number of ratings", ratings['rating'].count())

# number of different genres
genres = set()
for g in movies['genres'].dropna():
    
    genres.update(g.split("|"))
print("Number of different genres", len(genres))

# number of unique users
print("number of unique users", ratings['user_id'].nunique())
# number of missing ratings

print("number of missing ratings", sum(ratings['rating'].isna()))

# the `average`, `min`, and `max` ratings given
print("The `average`, `min`, and `max` ratings:",
      ratings['rating'].mean(),
      ratings['rating'].min(),
      ratings['rating'].max())

Number of movies 38013
number of ratings 921398
Number of different genres 28
number of unique users 71707
number of missing ratings 0
The `average`, `min`, and `max` ratings: 7.312627116620614 0 10


## Cleaning the data

### Movies
- Pull the date from the title and create new column
- Dummy the date column with 1's and 0's for each century of a movie (1800's, 1900's, and 2000's)
- Remove missing values
- Dummy column the genre with 1's and 0's for each genre

### Reviews
- Create a date out of time stamp

In [6]:
# Pull the date from the title and create new column
movies[['title','year']] = movies["title"].str.split(" \(",expand=True)
movies['year'] = movies['year'].str.replace("\)","")
movies.drop("movie_id",inplace = True, axis = 1)
movies.head()

<ipython-input-6-7290d973aa4e>:3: FutureWarning: The default value of regex will change from True to False in a future version.
  movies['year'] = movies['year'].str.replace("\)","")


,title,genres,year
0,Edison Kinetoscopic Record of a Sneeze,Documentary|Short,1894
1,La sortie des usines Lumière,Documentary|Short,1895
2,The Arrival of a Train,Documentary|Short,1896
3,The Oxford and Cambridge University Boat Race,NaN,1895
4,Le manoir du diable,Short|Horror,1896


In [7]:
# Split Genres

## Drop missing values
movies.dropna(inplace = True)

for gen in genres:
    movies[gen] = movies["genres"].apply(lambda x : 1 if gen in x else 0)
movies.head()

,title,genres,year,Comedy,Drama,Romance,Game-Show,War,Adult,Film-Noir,...,History,Sport,Crime,News,Biography,Western,Adventure,Horror,Action,Music
0,Edison Kinetoscopic Record of a Sneeze,Documentary|Short,1894,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,La sortie des usines Lumière,Documentary|Short,1895,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,The Arrival of a Train,Documentary|Short,1896,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Le manoir du diable,Short|Horror,1896,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
5,Une nuit terrible,Short|Comedy|Horror,1896,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0


In [8]:
# Create dummy columns for year 
movies['century'] = movies['year'].astype(int)//100 *100
pd.get_dummies(movies, columns = ['century']).head()

,title,genres,year,Comedy,Drama,Romance,Game-Show,War,Adult,Film-Noir,...,News,Biography,Western,Adventure,Horror,Action,Music,century_1800,century_1900,century_2000
0,Edison Kinetoscopic Record of a Sneeze,Documentary|Short,1894,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,La sortie des usines Lumière,Documentary|Short,1895,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,The Arrival of a Train,Documentary|Short,1896,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
4,Le manoir du diable,Short|Horror,1896,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0
5,Une nuit terrible,Short|Comedy|Horror,1896,1,0,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,0


In [9]:
# Save the dataframes to csv files
movies.to_csv("data/movies.csv", index = False)
ratings.to_csv("data/ratings.csv", index = False)
users.to_csv("data/users.csv", index = False)
